# Amazon Multipage Webscrape in Beautiful Soup

## This is for demonstration purposes only that yes we can programmatically scrape competitor price, product and rating data from Amazon, with the benefit of having no private API keys exposed on a public facing repository. But I caution Amazon aggressively monitors for and blacklists bots like this. 

## Better yet, the Amazon Product Advertising API using your seller account is the official API for data extraction on price, product reviews, etc. I can, and prefer, to work with these API's too!  

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
from fake_useragent import UserAgent
import pandas as pd

In [2]:
def get_soup_retry(url):
    '''This is to evade annoying bot interceptors'''
    ua= UserAgent()
    uag_rand= ua.random
    header={
        'user-agent': uag_rand,
        "accept-language": 'en-US'
    }
    isCaptcha = True
    while isCaptcha:
            page = requests.get(url, headers=header)
            assert page.status_code==200
            soup=BeautifulSoup(page.content, 'lxml')
            if 'captcha' in str(soup):
                uag_rand = ua.random
                print(f'\rbot detected..t=retrying..use new ua: {uag_rand}', end='', flush=True)
            else: 
                print('bot bypassed')
                return soup

def search_keyword(kw):
    count_page=0
    count_asin=0
    while True:
        count_page +=1
        url= f"https://www.amazon.ca/s?k={kw}&page={count_page}"
        print(f'Getting page: {count_page}|{url}')
        soup=get_soup_retry(url)
        result= soup.find('div', attrs={'class':'s-main-slot s-result-list s-search-results sg-row'}).find_all("div", attrs={'data-component-type': 's-search-result'})
        
        for ids in result:
            count_asin +=1
            asin = ids['data-asin']
            url=f'https://www.amazon.com/dp/{asin}'
            '''see below funtions for getting all the details on each item then write it to a df and to csv'''
            list_result = get_details(url)
            df=pd.DataFrame(list_result)
            df.to_csv(f'result_amazon.csv', index=False)
            
            
            print(f'{count_asin}.{url}')
            
        last_page = soup.find('li', attrs={'class':'a-disabled a-last'})
        if not last_page:
            pass
        else:
            break

result=[]

def get_details(url):
    soup = get_soup_retry(url)
    today = datetime.date.today()
    title = soup.find('span', attrs={'id':'productTitle'}).text.strip()
    try:
        price = soup.find('span', attrs={'class':"a-offscreen"}).text.strip() 
    except AttributeError:
        price=''
    try:
        review_count = soup.find('span', attrs={"id": 'acrCustomerReviewText'}).text.strip()
    except AttributeError:
        review_count='' 
    try:
        feature= soup.find("div", attrs={'id':'feature-bullets'}).find('ul', attrs={'class':'a-unordered-list a-vertical a-spacing-mini'}).find_all(('li'))
    
        sv_feature=[]
        for li in feature:
            text=li.find('span', attrs={'class':'a-list-item'})
            f=str(text.string).strip()
            if "None" in f:
                pass
            else:
                sv_feature.append(f)
    except AttributeError:
        sv_feature =''
    try:
        stock=soup.find('div', attrs={'id':"availability"}).find('span').text.strip()
    except AttributeError:
        stock ='' 
    try:
        description=soup.find('div', attrs={'id':"productDescription"}).text.strip()
    except AttributeError:
        description ='' 
        
    print(today)
    print(title)
    print(price)
    print(review_count)
    print(sv_feature)
    print(stock)
    print(description)
    
    goal={
        'datetime': today,
        'title':title,
        'price':price,
        'review_count': review_count,
        'features': sv_feature,
        'stock': stock,
        #'description': description
    }
    print(goal)
    result.append(goal)
    return result

In [ ]:
search_keyword('NVIDIA+graphics+card&crid')

In [ ]:
    '''
   Could be added below description in fun tion above. 
    
    dt_img = soup.select('#imageBlock_feature_div> script:nth-child(2)')
    try:
        script_text = dt_img[0].text
        json_str=re.search('{(.+)}', script_text)[0].replace("\'",'"').replace("null",'"null"')
        json_obj=json.loads(json_str)
        images_url=[]
        for i in json_obj['initial']:
            images_hires=i["hiRes"]
            images_large=i["large"]
            if images_hires is None:
                images_url.append(images_large)
            else:
                images_url.append(images_hires)
    except IndexErrorError:
        images_url=''
        

and add:
print(images_url)